# Library instalation

In [1]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 4.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 


# Imports

In [83]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from kaggle_environments import make, evaluate, utils

# Creating user agents

## 1

This agent always returns rock

In [3]:
%%writefile rock.py
ROCK = 0
PAPER = 1
SCISSORS = 2
def rock(observation, configuration):
  return ROCK

Writing rock.py


## 2

This agent always returns paper

In [4]:
%%writefile paper.py
ROCK = 0
PAPER = 1
SCISSORS = 2
def paper(observation, configuration):
  return PAPER

Writing paper.py


## 3

This agent always returns scissors

In [5]:
%%writefile scissors.py
ROCK = 0
PAPER = 1
SCISSORS = 2
def scissors(observation, configuration):
  return SCISSORS

Writing scissors.py


## 4

This agent returns random value from ROCK-value to SCISSORS-value

In [84]:
%%writefile random_agent.py
import random
ROCK = 0
PAPER = 1
SCISSORS = 2
def random_agent(observation, configuration):
  return random.randint(0, 2)

Overwriting random_agent.py


## 5

This agent sequently chages return value from ROCK to SCISSORS starting from ROCK

In [7]:
%%writefile roll_rps.py
roll_agent_last_step = 0
def roll_rps(observation, configuration):
  global roll_agent_last_step
  roll_agent_last_step += 1
  roll_agent_last_step = roll_agent_last_step % 3
  return roll_agent_last_step

Writing roll_rps.py


## 6

This agent sequently chages return value from ROCK to SCISSORS starting from random integer value

In [85]:
%%writefile roll_from_random.py
import random

roll_agent_last_step = random.randint(0, 2)

def roll_from_random(observation, configuration):
  global roll_agent_last_step
  roll_agent_last_step += 1
  roll_agent_last_step = roll_agent_last_step % 3
  return roll_agent_last_step

Overwriting roll_from_random.py


## 7

This agent copies opponent last action

In [9]:
%%writefile copy_opponent.py
import random

ROCK = 0
PAPER = 1
SCISSORS = 2

def copy_opponent(observation, configuration):
  if observation.step > 0:
      return observation.lastOpponentAction
  else:
      return random.randint(ROCK, SCISSORS)

Writing copy_opponent.py


## 8

This agent returns last opponent action plus one

In [10]:
%%writefile reaction_agent.py
import random

ROCK = 0
PAPER = 1
SCISSORS = 2

def reaction_agent(observation, configuration):
  if observation.step > 0:
      return observation.lastOpponentAction + 1
  else:
      return random.randint(ROCK, SCISSORS)

Writing reaction_agent.py


## 9

This agent returns None value

In [11]:
%%writefile none_agent.py
def none_agent(observation, configuration):
  return None

Writing none_agent.py


## 10

This agent is trying to find value, who has won the most times. If sucsessful - using it

In [86]:
%%writefile statistical.py
import random
from kaggle_environments.envs.rps.utils import get_score

ROCK = 0
PAPER = 1
SCISSORS = 2

last_statistical_action = None
opponents_moves = []

def statistical(observation, configuration):
  global opponents_moves
  global last_statistical_action
  if observation.step == 0:
    last_statistical_action = random.randint(ROCK, SCISSORS)
  elif observation.step > 0:
    score = get_score(last_statistical_action,
                      observation.lastOpponentAction)
    if score == 1:
      opponents_moves.append(observation.lastOpponentAction)
      results = {i:opponents_moves.count(i)
                 for i in range(ROCK, SCISSORS)}
      results = sorted(results.items(), key=lambda item: item[1],
                        reverse=True)
      last_statistical_action = results[0][0]
    else:
      last_statistical_action = observation.lastOpponentAction + 1
  return last_statistical_action

Overwriting statistical.py


## 11

This agent takes one of two values: ROCK and SCISSORS randomly

In [13]:
%%writefile random_rs.py
import random

ROCK = 0
PAPER = 1
SCISSORS = 2

def random_rs(observation, configuration):
  next_move = random.randint(ROCK, PAPER)
  next_move = ROCK if next_move == 0 else SCISSORS
  return next_move

Writing random_rs.py


## 12

This agent returns sum of its and opponents last actions

In [79]:
%%writefile sum_agent.py
import random

ROCK = 0
PAPER = 1
SCISSORS = 2

agent_last_action = ROCK

def sum_agent(observation, configuration):
  global agent_last_action
  if observation.step == 0:
    next_action = random.randint(ROCK, SCISSORS)
  elif observation.step > 0:
    next_action = agent_last_action + observation.lastOpponentAction
  return next_action

Overwriting sum_agent.py


## Simulation functions

In [62]:
def get_reward(env, player):
  reward = env.state[0]['observation']['reward']
  return reward

def get_steps(env):
  steps = env.state[0]['observation']['step']
  return steps

def do_simulation(agents, do_visualise=True):
  cols = ['first_player', 'first_player_reward',
           'second_player', 'second_player_reward',
           'game_steps']
  results_dict = {col:[] for col in cols}
  for i in range(len(agents)):
    first_player_idx = random.randint(0, agents_amount - 1)
    second_player_idx = random.randint(0, agents_amount - 1)
    first_player = agents[first_player_idx] + '.py'
    second_player = agents[second_player_idx] + '.py'

    env.reset()
    # print(first_player, second_player)
    try:
      env.run([first_player, second_player])
    except Exception as e:
      print(e)
      pass

    results_dict['first_player'].append(first_player)
    results_dict['first_player_reward'].append(get_reward(env, 0))
    results_dict['second_player'].append(second_player)
    results_dict['second_player_reward'].append(get_reward(env, 1))
    results_dict['game_steps'].append(get_steps(env))
    if do_visualise:
      env.render(mode="ipython", width=700, height=550)
  game_results = pd.DataFrame(results_dict, columns=cols)
  return game_results

# Simulation

If `do_visualise` is True simulation will render video

In [89]:
agents = ['rock', 'paper', 'scissors', 'random_agent', 'roll_rps', 
          'roll_from_random', 'copy_opponent', 'reaction_agent',
          'none_agent', 'statistical', 'random_rs', 'sum_agent']
agents_amount = len(agents)
env = make("rps", debug=True, configuration={"episodeSteps": 50})
env.render()
do_simulation(agents, do_visualise=False)


,first_player,first_player_reward,second_player,second_player_reward,game_steps
0,reaction_agent.py,1,roll_from_random.py,1,3
1,sum_agent.py,-49,roll_from_random.py,-49,49
2,none_agent.py,-49,paper.py,-49,49
3,roll_rps.py,48,sum_agent.py,48,49
4,scissors.py,0,statistical.py,0,2
5,scissors.py,0,scissors.py,0,49
6,reaction_agent.py,-1,copy_opponent.py,-1,2
7,reaction_agent.py,1,sum_agent.py,1,4
8,sum_agent.py,0,sum_agent.py,0,49
9,copy_opponent.py,0,none_agent.py,0,49
